# Pure State Tomography

### Junling Long, Adrian Tan, Sven Jandura

# Introduction

 Suppose we have some quantum circuit, which generates a state |psi>. We are interested in the coefficients of psi.
 Unfortunatly, due to noise effects, on a real quantum computer we can only generate rho = (1-epsilon)|psi><psi| + epsilon rho_error
 with epsilon << 1. How do we find a good estimate of |psi>

 Using usual quantum state tomography, we can could find rho and would estimate |psi> to be the eigenvector of rho which belongs to
 the larges eigenvalue. Using pure state tomography, we can find an estimate of |psi> directly from the measurements. This requires 
 fewer operators to be measured then for a full state tomography for the same accuraccy.


# Import

In [1]:
import numpy as np

import qiskit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise
import qiskit.ignis.verification.tomography as tomo

from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise

from qiskit.ignis.verification.tomography.fitters.pure_state_mle_fit import pure_state_mle_fit, pure_state_mle_fit_density_matrix

from scipy.linalg import sqrtm


/usr/lib/python3.7/site-packages/marshmallow/schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


# Testing the protocol with simplified noise model

## 1. Generating test density matrix

Consider the Bell states |psi1> = 1/sqrt(2)(|00>+|11>) and |psi2> = 1/sqrt(2)(|00>-|11>) and rho = (1-epsilon)|psi1><psi1| + epsilon|psi2><psi2|



In [2]:
#Psi1
# Setup circuit to generate |psi1>
qr1 = QuantumRegister(2)
bell1 = QuantumCircuit(qr1)
bell1.h(qr1[0])
bell1.cx(qr1[0], qr1[1])

# get state |psi1> using statevector_simulator
job = qiskit.execute(bell1, Aer.get_backend('statevector_simulator'))
psi1 = job.result().get_statevector(bell1)
print("|psi1> = {}".format(psi1))

# get expectation values of operators ... and convert the to the right format
qst_bell1 = tomo.state_tomography_circuits(bell1, qr1)
job = qiskit.execute(qst_bell1, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi1 = tomo.tomography_data(job.result(), qst_bell1, efficient = True)
probs_psi1, basis_matrix, _ = tomo.fitter_data(tomo_counts_psi1)

#Psi 2
# Setup circuit to generate |psi2>
qr2 = QuantumRegister(2)
bell2 = QuantumCircuit(qr2)
bell2.y(qr2[0])
bell2.h(qr2[0])
bell2.cx(qr2[0], qr2[1])

# get state |psi2> using statevector_simulator
job = qiskit.execute(bell2, Aer.get_backend('statevector_simulator'))
psi2 = job.result().get_statevector(bell2)
print("|psi2> = {}".format(psi2))


# get expectation values of operators ... and convert the to the right format
qst_bell2 = tomo.state_tomography_circuits(bell2, qr2)
job = qiskit.execute(qst_bell2, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi2 = tomo.tomography_data(job.result(), qst_bell2, efficient = True)
probs_psi2, _ , _ = tomo.fitter_data(tomo_counts_psi2) # the basis_matrix is the same for both states


|psi1> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|psi2> = [0.+0.70710678j 0.+0.j         0.+0.j         0.-0.70710678j]


Generate probabilities of rho

In [3]:
epsilon = 0.3
probs_rho = [(1-epsilon)*probs_psi1[i]+epsilon*probs_psi2[i] for i in range(len(probs_psi1))]

function for calculating inner product between states


In [4]:
def inner_product(psi, phi):
    return np.abs(sum([psi[i].conj()*phi[i] for i in range(len(psi))]))**2

## 2. Unconstrained state tomography (maximum likelyhood approach)


In [5]:
rho_full_mle = tomo.state_mle_fit(probs_rho, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi1, eigenvectors[0])))

Eigenvalues of rho: [ 5.66746117e-01+8.33213245e-19j  3.66602386e-01+5.23432526e-19j
 -1.48442426e-18+7.00058918e-19j  6.66514964e-02-1.18934295e-18j]
Guess for psi: [ 6.93738235e-01-0.00330093j  7.20200176e-01+0.j
  8.55996862e-04-0.00423786j -7.05250544e-04-0.00298034j]
Fidelity: 0.24016708574183632


## 3. Pure state tomography (maximum likelyhood approach)


In [6]:
psi_guess,_ = pure_state_mle_fit(probs_rho, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi1, psi_guess)))


Guess for psi: [(0.7041157488576673+0.0005531184141830959j), (0.004316117488619866-0.003419005310224335j), (0.0003417032354682942-0.0025189438227530576j), (0.7100483171842213+0.0039132119686459315j)]
Fidelity: 0.9999399768956078


# Testing the protocol with standard noise model

## 1. Set up random circuit

In [7]:
qr = QuantumRegister(2)
circuit = QuantumCircuit(qr)
params = np.random.rand(12)
circuit.u3(params[0], params[1], params[2], qr[0])
circuit.u3(params[3], params[4], params[5], qr[1])
circuit.cx(qr[0], qr[1])
circuit.u3(params[6], params[7], params[8], qr[0])
circuit.u3(params[9], params[10], params[11], qr[1])


job = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(circuit)
print("psi: {}".format(psi))

psi: [0.83832559+0.00067694j 0.38198031-0.00400993j 0.10831549+0.00098786j
 0.1381847 +0.34706842j]


## 2. Apply standard noise

In [8]:
#IBMQ.load_accounts()
IBMQ.enable_account('a6140115a9d2692b8a711d0f31fdc92b7ae793865719445a78fc210220de52765da18d0eab774d245fbc9e5f40c94e99d9c536f9f1749acd7b902b3bd9931dd5')
device = IBMQ.get_backend('ibmqx4')
properties = device.properties()
coupling_map = device.configuration().coupling_map

gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678)]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)


# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')


## 3. Execute

In [9]:
qst= tomo.state_tomography_circuits(circuit, qr)
job = qiskit.execute(qst, Aer.get_backend('qasm_simulator'),noise_model=noise_model, coupling_map=coupling_map, basis_gates=basis_gates, shots=5000)
tomo_counts = tomo.tomography_data(job.result(), qst, efficient = True)
probs_psi, basis_matrix, _ = tomo.fitter_data(tomo_counts)

## 4. Unconstrained state tomography (maximum likelyhood approach)


In [10]:
rho_full_mle = tomo.state_mle_fit(probs_psi, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi, eigenvectors[0])))

Eigenvalues of rho: [0.4791974 +1.92839891e-18j 0.09497703-3.53426719e-19j
 0.25105974-5.39893583e-19j 0.17476583-2.87906542e-17j]
Guess for psi: [ 0.96053798+0.j         -0.01619195+0.07615421j -0.07929213+0.22442906j
 -0.07235255+0.09702896j]
Fidelity: 0.671032396552017


## 5. Pure state tomography (maximum likelyhood approach)

In [11]:
psi_guess,_ = pure_state_mle_fit(probs_psi, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi, psi_guess)))

Guess for psi: [(0.8092028547683698-0.039007297651932574j), (0.44953552968474525-0.013402981540578993j), (0.12746041881919218-0.0022139396218459817j), (0.14087664601057778+0.3245150981327211j)]
Fidelity: 0.9937278224214127
